# CNP_func_gridsearch_age.ipynb

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVR
# from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from nilearn.input_data import NiftiMasker
import random
import matplotlib.pyplot as plt
import datetime

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

In [2]:
# Parameters
# Config
toy = False
run_all = False

path = '/home/ubuntu/fmriprep/'

#AWS cloud
input_dir = '../output/' #AWS cloud
output_dir = '../output/' 

#For Daniel's computer
# input_dir = 'data/CNP_func/' 
# output_dir = '/output/' 

n_jobs = 20 #amount of cores
cv=4

parcellation = '444'
description = 'CNP_func_gridsearch_age_first_'+parcellation
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [3]:
loaded = np.load(input_dir+'CNP_func_gridsearch_age_18-08-09-22-05-36CNP_func'+parcellation+'_train_set.npz')
X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']


In [4]:

print(X_train.shape)
X_train = np.reshape(X_train, (84, X_train.shape[2]))

(84, 1, 98346)


## Gridsearch 
### Models
* Linear Support vector regressor
    * C = 0.0001, 0.001, 0.1, 1.0, 10.0

### Feature selection
* ANOVA
    * 5k features
    * 100k features
    * Use all features (~1.9m)

In [5]:
from sklearn.pipeline import Pipeline

if toy:
    X_train = X_train[:4]
    y_train = y_train[:4]


# Remove features with too low between-subject variance
# variance_threshold = 

# Here we use a classical univariate feature selection based on F-test,
# namely Anova.
# feature_selection = SelectKBest(f_regression)

# We have our predictor (SVR), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
               
            
anova_svr = Pipeline([
    ('anova', SelectKBest(f_regression)),
    ('svr', LinearSVR())])

parameters = [{'anova__k': [1000, 'all'],
               'svr__C': [0.001, 0.01, 1, 10]}]

if toy: 
    parameters = [{'anova__k': [5000],'svr__C': [0.00001, 0.0001]}]


grid = GridSearchCV(anova_svr, cv=cv, n_jobs=n_jobs, param_grid=parameters, scoring='neg_mean_absolute_error')
grid.fit(X_train, y_train)

# mean_scores = np.array(grid.cv_results_['neg_mean_absolute_error'])
# mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# # select score for best C
# mean_scores = mean_scores.max(axis=0)
# bar_offsets = (np.arange(len(N_FEATURES_OPTIONS)) *
#                (len(reducer_labels) + 1) + .5)






GridSearchCV(cv=4, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('anova', SelectKBest(k=10, score_func=<function f_regression at 0x7f09544c67b8>)), ('svr', LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0))]),
       fit_params=None, iid=True, n_jobs=20,
       param_grid=[{'svr__C': [0.001, 0.01, 1, 10], 'anova__k': [1000, 'all']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [6]:
print('best score: ',grid.best_score_)
print('best params: ',grid.best_params_)
print('\n\nfullresults: \n', grid.cv_results_)
df1 = pd.DataFrame(grid.cv_results_)
df1

best score:  -8.327798909386647
best params:  {'svr__C': 1, 'anova__k': 'all'}


fullresults: 
 {'std_train_score': array([8.70350537e-01, 1.08333647e+00, 6.00962628e-02, 2.02629049e-04,
       3.48205455e-01, 1.44041899e-02, 2.40682176e-04, 2.40682176e-04]), 'std_score_time': array([0.0030555 , 0.00627212, 0.01574537, 0.00978136, 0.02455513,
       0.00980281, 0.01221563, 0.00891216]), 'split0_train_score': array([-2.50162306e+01, -1.30567062e+01, -5.71901887e-03, -1.23309074e-03,
       -6.20736627e+00, -9.56163061e-02, -8.41165647e-04, -8.41165647e-04]), 'std_fit_time': array([0.05261286, 0.02024082, 0.04905604, 0.02926576, 0.18716387,
       0.36862665, 0.65602003, 0.47933952]), 'split3_train_score': array([-2.54173195e+01, -1.10186539e+01, -1.54953756e-02, -6.68564370e-04,
       -6.04300302e+00, -1.12180178e-01, -1.38101053e-03, -1.38101053e-03]), 'split3_test_score': array([-24.76978779, -11.39027319,  -8.48035   ,  -8.46218647,
        -9.6950854 ,  -8.59978956,  -8.60297907,  

/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-pac

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_anova__k,param_svr__C,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,split3_test_score,split3_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.291870,0.023232,-24.860567,-24.801149,1000,0.001,"{'svr__C': 0.001, 'anova__k': 1000}",8,-24.297124,-25.016231,-26.252606,-23.322832,-24.122750,-25.448215,-24.769788,-25.417319,0.052613,0.003055,0.837833,0.870351
1,0.257660,0.020846,-12.318107,-11.648567,1000,0.01,"{'svr__C': 0.01, 'anova__k': 1000}",7,-12.064576,-13.056706,-13.838519,-12.262451,-11.979058,-10.256457,-11.390273,-11.018654,0.020241,0.006272,0.915391,1.083336
2,0.337619,0.039802,-8.809213,-0.049711,1000,1,"{'svr__C': 1, 'anova__k': 1000}",5,-7.867889,-0.005719,-8.772415,-0.153165,-10.116199,-0.024464,-8.480350,-0.015495,0.049056,0.015745,0.822163,0.060096
3,0.332052,0.026115,-8.752382,-0.000918,1000,10,"{'svr__C': 10, 'anova__k': 1000}",4,-7.868705,-0.001233,-8.537160,-0.000868,-10.141479,-0.000903,-8.462186,-0.000669,0.029266,0.009781,0.842765,0.000203
4,1.209372,0.058093,-9.201233,-5.784296,all,0.001,"{'svr__C': 0.001, 'anova__k': 'all'}",6,-7.301250,-6.207366,-10.012098,-5.501120,-9.796498,-5.385696,-9.695085,-6.043003,0.187164,0.024555,1.102913,0.348205
5,7.039674,0.055966,-8.344133,-0.091395,all,0.01,"{'svr__C': 0.01, 'anova__k': 'all'}",3,-6.416289,-0.095616,-8.861233,-0.084698,-9.499219,-0.073086,-8.599790,-0.112180,0.368627,0.009803,1.160125,0.014404
6,7.145573,0.032499,-8.327799,-0.000966,all,1,"{'svr__C': 1, 'anova__k': 'all'}",1,-6.438442,-0.000841,-8.747708,-0.000851,-9.522067,-0.000791,-8.602979,-0.001381,0.656020,0.012216,1.145425,0.000241
7,6.740567,0.021232,-8.327799,-0.000966,all,10,"{'svr__C': 10, 'anova__k': 'all'}",1,-6.438442,-0.000841,-8.747708,-0.000851,-9.522067,-0.000791,-8.602979,-0.001381,0.479340,0.008912,1.145425,0.000241


In [7]:
age_pred = grid.predict(X_train)
mean_abs_error = np.mean(np.abs(age_pred-y_train))
df1 = pd.DataFrame(grid.cv_results_)

with open(output_dir+'log_'+log_file+'.txt', '+a') as f:
    f.write('best score: '+str(grid.best_score_))
    f.write('best params: '+str(grid.best_params_))
    f.write('\n\nfullresults: \n'+str(grid.cv_results_)+'\n')
    f.write(str(df1)+'\n')
    f.write(str(age_pred)+'\n')
    f.write(str(y_train)+'\n')
    f.write(str(mean_abs_error)+'\n')
    

/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-pac